If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
#! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input your username and password (this only works on Colab, in a regular notebook, you need to do this in a terminal):

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/matt/.huggingface/token


Then you need to install Git-LFS and setup Git if you haven't already. On Linux, uncomment the following instructions and adapt with your name and email. On Windows, please download git-lfs at https://git-lfs.github.com/

In [2]:
# !apt install git-lfs
# !git config --global user.email "you@example.com"
# !git config --global user.name "Your Name"

Make sure your version of Transformers is at least 4.8.1 since the functionality was introduced in that version:

In [3]:
import transformers

print(transformers.__version__)

4.11.0.dev0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

# Train a language model

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to train a model on it.

This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook ([open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)).

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [4]:
from datasets import load_dataset

datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

Reusing dataset wikitext (/home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


  0%|          | 0/3 [00:00<?, ?it/s]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [5]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [6]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,text
0,
1,"Our Darling ’ s First Book ( written in collaboration with Dorothy Barker ) ; Blackie , 1929 \n"
2,"Little is known of the ancestry of the Yorkshire branch of the Calverts . At George Calvert 's knighting , it was claimed that his family originally came from Flanders ( a Dutch @-@ speaking area today across the English Channel in modern @-@ day Kingdom of Belgium ) . Calvert 's father , ( an earlier ) Leonard was a country gentleman who had achieved some prominence as a tenant of Lord Wharton , and was wealthy enough to marry a "" gentlewoman "" of a noble line , Alicia or Alice Crossland ( or sometimes spelled : "" Crosland "" ) . He established his family on the estate of the later @-@ built Kiplin Hall , near Catterick in Richmondshire , of Yorkshire . George Calvert was born at Kiplin in late 1579 ( birth month and day yet to be researched ) . His mother Alicia / Alice died on 28 November 1587 , when he was fifteen years old . His father then married Grace Crossland ( or sometimes spelled : "" Crosland "" ) , Alicia 's first cousin . \n"
3,"From 1986 to 1992 , Deal was a member of the Pixies , and from 1989 onwards , the Breeders . In August 1993 , the Breeders released their second album , Last Splash , which went platinum in the USA , gold in Canada , and silver in the UK . The other members of the group at that time were Kim 's twin sister Kelley Deal , Josephine Wiggs and Jim Macpherson . By late 1994 , after two years of straight touring and recording , and culminating in the Lollapalooza tour , the band members were exhausted ; they decided to take some time off from the Breeders , but this hiatus ended up being longer than expected . Kelley was arrested on drug charges in late 1994 and spent time in and out of rehabilitation , while Wiggs became involved in musical projects in New York , including collaborations with members of Luscious Jackson . \n"
4,"In 1965 and 1966 , Dylan was touring with the Hawks — Rick Danko , Garth Hudson , Richard Manuel , Robbie Robertson and Levon Helm , although Helm quit the group in late November or early December 1965 . In July 1966 , Dylan suffered a motorcycle accident and spent several months recuperating at his house in Byrdcliffe , near Woodstock , New York . By spring 1967 , all of the members of the Hawks , except Helm , had joined Dylan in the Woodstock area , with Danko , Manuel and Hudson living in nearby West Saugerties in a house nicknamed Big Pink . Dylan and the four Hawks began recording informal music sessions , first at Dylan 's house in what was known as "" the Red Room "" , followed by the basement of Big Pink . Earlier on they recorded mostly covers and traditional music , but later moved onto original material written largely by Dylan . In total , over 100 songs and alternate takes were put on tape . Helm returned to the group in October 1967 and performed on some final Woodstock @-@ area collaborations between Dylan and the Hawks , these ones at a different house that some group members had moved to . In the fall of that year , the Hawks , who soon renamed themselves the Band , continued writing and rehearsing songs for their debut album , Music From Big Pink . \n"
5,
6,= = = Christianity = = = \n
7,
8,"After the Falkland Islands Dependencies Survey was renamed British Antarctic Survey in 1962 , a series of depots were developed , including one at Mount Jackson . BAS members were successful in making many first ascents of the mountains of the Antarctic Peninsula , and John Cunningham , who served at the BAS base at Adelaide Island , was the first to climb Mount Jackson . Traveling by dog sled for 640 kilometres ( 400 mi ) , his team reached the summit on 23 November 1964 . \n"
9,


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead. For the tokenizer, you can replace the checkpoint by the one you trained yourself.

In [9]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [12]:
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)

Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-74ff3ff1409c85ed.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-2319e2bfeb5df81e.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-3ebdcaa2a2d7e629.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-a1172625ffd1c5d7.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-babc003593767

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [13]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [238, 8576, 9441, 2987, 238, 252]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [14]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-7d2ae288826da460.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-25d853c30fc66f4f.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-8d12ae69b6b0d73e.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-21bc14e654401a14.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20/cache-2442dd65124e5

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'

Now that the data has been cleaned, we're ready to instantiate our `Model`. First we create the model using the same config as our checkpoint, but initialized with random weights:

In [18]:
from transformers import AutoConfig, TFAutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = TFAutoModelForCausalLM.from_config(config)

2021-09-26 17:03:20.511573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 17:03:20.515572: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 17:03:20.516257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-26 17:03:20.517429: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Now let's set some hyperparameters like the learning rate and weight decay, as well as the model ID, if we want to upload our model to the Hub afterwards.

In [19]:
learning_rate = 2e-5
weight_decay = 0.01
push_to_hub_model_id = f"{model_checkpoint}-wikitext2"

Now we initialize our optimizer.

In [20]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

All Transformers models compute loss internally, so we can just use a 'dummy' loss function that passes that value through and then compile our model with that loss on the 'loss' output head.

In [21]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. To disable this behaviour, please explicitly pass loss=None.


Finally, we need to convert our datasets to a format Keras understands. The easiest way to do this is with the `to_tf_dataset()` method.

In [22]:
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
)
validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
)

And we can train our model:

In [23]:
model.fit(train_set, validation_data=validation_set, epochs=2)

2021-09-26 17:03:30.655210: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
   1/1124 [..............................] - ETA: 3:13:03 - loss: 10.9455

2021-09-26 17:03:40.873634: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1124/1124 [==============================] - 189s 159ms/step - loss: 7.3081 - val_loss: 6.7675
Epoch 2/2
1124/1124 [==============================] - 179s 159ms/step - loss: 6.4943 - val_loss: 6.3513


Once the training is completed, we can evaluate our model and get its loss on the validation set like this:

In [24]:
eval_loss = model.evaluate(validation_set)

121/121 [==============================] - 7s 56ms/step - loss: 6.3513


The quality of language models is often measured in 'perplexity' rather than cross-entropy. To convert to perplexity, we simply raise e to the power of the cross-entropy loss.

In [25]:
import math

print(f"Perplexity: {math.exp(eval_loss):.2f}")

Perplexity: 573.25


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction.

In [26]:
model.push_to_hub(push_to_hub_model_id)

Upload file tf_model.h5:   0%|          | 32.0k/475M [00:00<?, ?B/s]

To https://huggingface.co/huggingface/gpt2-wikitext2
   0dd6aa4..aadd25d  main -> main



'https://huggingface.co/huggingface/gpt2-wikitext2/commit/aadd25db0acef6243d4c293c344cb170d9921148'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained("your-username/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!

We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead. For the tokenizer, replace the checkpoint by the one you trained.

In [27]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [28]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [29]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [30]:
from transformers import AutoConfig, TFAutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = TFAutoModelForMaskedLM.from_config(config)

/home/matt/miniconda3/envs/tensorflow26/lib/python3.9/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


We redefine our hyperparameters and choose a new name:

In [31]:
learning_rate = 2e-5
weight_decay = 0.01
push_to_hub_model_id = f"{model_checkpoint}-wikitext2"

Now we initialize our optimizer.

In [32]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

All Transformers models compute loss internally, so as in the CLM example we can just leave the loss argument blank to use the internal loss.

In [33]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. To disable this behaviour, please explicitly pass loss=None.


Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking. Make sure to set `return_tensors="tf"` too - the `DataCollator` objects all support multiple frameworks, and we don't want to accidentally get a bunch of `torch.Tensor` objects floating around in our TensorFlow code!

In [34]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf"
)

Now we pass our data collator to the `to_tf_dataset()` argument.

In [35]:
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

And now we can train our model:

In [36]:
model.fit(train_set, validation_data=validation_set, epochs=2)

Epoch 1/2
1172/1172 [==============================] - 173s 139ms/step - loss: 7.0812 - val_loss: 6.5040
Epoch 2/2
1172/1172 [==============================] - 161s 137ms/step - loss: 6.4081 - val_loss: 6.2692


Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [37]:
eval_loss = model.evaluate(validation_set)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

126/126 [==============================] - 5s 43ms/step - loss: 6.2595
Perplexity: 522.97


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [38]:
model.push_to_hub(push_to_hub_model_id)

Upload file tf_model.h5:   0%|          | 32.0k/500M [00:00<?, ?B/s]

To https://huggingface.co/huggingface/bert-base-cased-wikitext2
   e02388a..b881653  main -> main



'https://huggingface.co/huggingface/bert-base-cased-wikitext2/commit/b8816534f8cca663e3ace39a84dc0bfd845e876a'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import TFAutoModelForMaskedLM

model = TFAutoModelForMaskedLM.from_pretrained("your-username/my-awesome-model")
```